## Campus Devcon - FEU Tech

### Sample Implementations of OpenAI Projects

In [4]:
import os
import requests

import openai
from transformers import CLIPProcessor, CLIPModel

from IPython.display import Image
from IPython.core.display import HTML 

import PIL

import numpy as np

openai.api_key = os.getenv("OPENAI_API_KEY")

### Load some functions

In [5]:
def generate_image(prompt = ""):

    response = openai.Image.create(
      prompt = prompt,
      n=1,
      size="256x256"
    )
    
    image_url = response['data'][0]['url']

    return image_url


def generate_image_variation(image_location):

    response = openai.Image.create_variation(
      image=open(image_location, "rb"),
      n=1,
      size="256x256"
    )
    image_url = response['data'][0]['url']   

    return image_url

def edit_image(image_location, prompt):

    response = openai.Image.create_edit(
        image = open(image_location, "rb"),
        mask = open("data/transparent_mask.png", "rb"),
        prompt= prompt,
        n=1,
        size="256x256"
    )
    image_url = response['data'][0]['url']
    return image_url

def load_raw_image(image_location):
    return PIL.Image.open(image_location)

def save_image(image_url, file_location):
    
    img_data = requests.get(image_url).content
    
    with open(file_location, 'wb') as handler:
        handler.write(img_data)

def get_image(image_url):
    return Image(url= image_url)

## Generate Images

In [56]:
dog_url = generate_image(prompt = "playful dog")
get_image(image_url = dog_url)

In [57]:
save_image(image_url = dog_url, file_location = "data/dog.png")

In [58]:
another_dog_url = generate_image_variation(image_location = "data/dog.png")
get_image(image_url = another_dog_url)

In [59]:
save_image(image_url = another_dog_url, file_location = "data/dog1.png")

In [60]:
dog_and_cat_url = generate_image(prompt = "dog and cat")
get_image(image_url = dog_and_cat_url)

In [61]:
save_image(image_url = dog_and_cat_url, file_location = "data/cat_and_dog.png")

In [62]:
edited_cat_and_dog_url = edit_image(
    image_location = "data/cat_and_dog.png", 
    prompt = "dog and cat are playing in garden")
get_image(image_url = edited_cat_and_dog_url)

In [63]:
save_image(image_url = dog_and_cat_url, file_location = "data/cat_and_dog_garden.png")

## Pretrained Models

## Description

The CLIP model was developed by researchers at OpenAI to learn about what contributes to robustness in computer vision tasks. The model was also developed to test the ability of models to generalize to arbitrary image classification tasks in a zero-shot manner. It was not developed for general model deployment - to deploy models like CLIP, researchers will first need to carefully study their capabilities in relation to the specific context they’re being deployed within.

The base model uses a ViT-L/14 Transformer architecture as an image encoder and uses a masked self-attention Transformer as a text encoder. These encoders are trained to maximize the similarity of (image, text) pairs via a contrastive loss.

The original implementation had two variants: one using a ResNet image encoder and the other using a Vision Transformer. This repository has the variant with the Vision Transformer.

See more: https://huggingface.co/openai/clip-vit-large-patch14

In [16]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [64]:
dog_image = load_raw_image("data/dog1.png")

In [18]:
def get_prediction(image, labels):

    inputs = processor(
        text           = labels, 
        images         = image, 
        return_tensors = "pt", 
        padding        = True
    )

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim = 1) 
    probs = probs.detach().cpu().numpy().tolist()[0]

    results = dict(zip(labels,probs))

    return results

In [65]:
get_prediction(image = dog_image, labels = ["a photo of a cat", "a photo of a dog"])

{'a photo of a cat': 0.0036707331892102957,
 'a photo of a dog': 0.9963292479515076}

In [66]:
get_prediction(image = dog_image, labels = ["dog", "cat"])

{'dog': 0.9992706179618835, 'cat': 0.0007293628877960145}

In [67]:
catdog_image = load_raw_image("data/cat_and_dog.png")
get_prediction(image = catdog_image, labels = ["dog", "cat"])

{'dog': 0.6017462611198425, 'cat': 0.39825376868247986}

## Natural language Processing

In [22]:
statement = """
  {0}
  
  I never imagined that I would find myself in a situation where I have to explain and defend my decision to leave a marriage, as well as justify the state of my mental health.

  It is truly disheartening that I am being questioned and asked to prove my worth as a songwriter and artist. However, despite the overwhelming circumstances, I feel a strong inner calling to stand up for myself and protect my integrity.

  I want to set things straight once and for all. Let me be clear: I have never employed a ghostwriter. Throughout my career, I have been fortunate enough to collaborate with immensely talented artists who treat each other with respect and acknowledge the contributions we all bring to the table. 

  Every song I have created is a true reflection of my deepest emotions and experiences. I am fully prepared to provide evidence that supports this truth, including screenshots of conversations and recordings of my songs, which will undoubtedly validate and disprove the harmful accusations made against me.

  I did not cheat on Jason. It is unfair that I find myself in a position where I have to defend my name and even explain why I did not deserve to be cheated on.

  Currently, my focus is on the path towards healing, as the weight of trauma still lingers heavily in my heart. This process is not only essential for my own well-being but also for the well-being of those dear to me who have unwaveringly stood by my side throughout these challenging times-my loving family, my dedicated band, my friends, who have all been my source of emotional support.

  Their presence has been my anchor, grounding me during these turbulent moments. I hold onto hope that, one day, when the dust settles and clarity is restored, I will have the capacity to embrace a love that is faithful and genuine. I take solace in the knowledge that I am Moira-an artist-who will bravely navigate these challenges and emerge stronger than ever.
"""

In [68]:
sentiment_analysis_prompt = statement.format("Classify the sentiment of the following statement:")

In [69]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= sentiment_analysis_prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

response["choices"][0]["text"]

'\nThe sentiment of this statement is hopeful and determined.'

In [70]:
sentiment_analysis_prompt = statement.format("Classify as positive or negative sentiment the following statement:")

In [71]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= sentiment_analysis_prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

response["choices"][0]["text"]

'\nNegative sentiment.'

In [73]:
summarize_prompt = statement.format("Summarize the following statement into strictly 2-3 sentences:")

In [74]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= summarize_prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

response["choices"][0]["text"]

'\nI am facing difficult circumstances as I have to explain and defend my decision to leave a marriage and justify my mental health. I am determined to stand up for myself and protect my integrity, and I am prepared to provide evidence that I have never employed a ghostwriter. I am focusing on healing and leaning on the support of my loving family, band, and friends. I am hopeful that I will be able to embrace a faithful and genuine love in the future.'

In [75]:
generate_image_code = """
    {0}
    
    def generate_image(prompt = ""):
    
        response = openai.Image.create(
          prompt = prompt,
          n=1,
          size="256x256"
        )
        
        image_url = response['data'][0]['url']
    
        return image_url
"""

In [76]:
code_prompt = generate_image_code.format("Annotate the following code:")

In [77]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt = code_prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(response["choices"][0]["text"])


This function takes in an optional prompt string as an argument and uses the OpenAI Image API to generate an image with a size of 256x256. It then returns the URL of the generated image.


In [78]:
code_prompt = generate_image_code.format("Update the following code with docstring:")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt = code_prompt,
  temperature=0,
  max_tokens=250,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(response["choices"][0]["text"])

    
def generate_image(prompt = ""):
    """
    Generates an image based on the given prompt.
    
    Parameters
    ----------
    prompt : str
        The prompt to generate the image from.
    
    Returns
    -------
    image_url : str
        The URL of the generated image.
    """
    response = openai.Image.create(
      prompt = prompt,
      n=1,
      size="256x256"
    )
    
    image_url = response['data'][0]['url']

    return image_url


### Final Code

In [33]:
def generate_image(prompt = ""):
    """
    Generates an image based on the given prompt.
    
    Parameters
    ----------
    prompt : str
        The prompt to generate the image from.
    
    Returns
    -------
    image_url : str
        The URL of the generated image.
    """
    response = openai.Image.create(
      prompt = prompt,
      n=1,
      size="256x256"
    )
    
    image_url = response['data'][0]['url']

    return image_url

In [38]:
prompt = statement.format("Extract keywords from following statement and create a prompt to create image based on the top 2 derived keywords:")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

sentiment_prompt = response["choices"][0]["text"]

print(sentiment_prompt)

image_from_sentiment_url = generate_image(prompt = sentiment_prompt)
get_image(image_url = image_from_sentiment_url)


Keywords: Healing, Strength, Love, Faithful, Genuine, Bravely Navigate

Prompt: Showing strength and resilience in the face of adversity, create an image that captures the journey of healing and finding love that is faithful and genuine.


In [43]:
statement_2 = """
   {0}

   Breaking my silence about the Jason-Moira breakup.

Warning: long post ahead. Kung maiksi attention span mo, 'wag mo na basahin.

Una sa lahat, I will share only what "I believe I know" about the story, based na rin sa personal experiences ko with the ex-couple.

Moira used to be a friend to me. Absentee ninang ni Ludwig. She was proxied by her whole family nung binyag. We were from the same city--Olongapo. But we only got to know more about each other during the grand finals of Himig Handog 2017, kung saan naging grand winner ang kantang Titibo-tibo, na personal song namin ni Libertine. Doon na rin namin nakilala si Jason at iba pang malapit sa buhay nila. After Titibo-tibo, nagkaroon pa kami ng ilang song collaborations ni Moira. The most recent was the song for presidential bet Leni Robredo. Mind you, I was not paid a single penny for the songs I wrote. There was an instance pa nga na I was not even included in the credits sa Spotify. Kung hindi ko pa pinuna, hindi nya pa ire-rectify. (Rhyme intended.)

May iba pang pagkakataon na nahahalata kong there's something about Moira. It could be a mental health condition, I'm not sure. I could be wrong. Pero nahahalata ko na napapadalas ang pagsisinungaling nya. Kung sa akin nagagawa nya ito, ano pa kaya sa mga taong mahal na mahal sya. (I have proofs in case I'll be asked to produce them.) Iniisip ko na lang, baka epekto ito ng stress dahil sa napakahectic nyang schedule. Or baka side effects ng mga meds nya for her conditions--pcos, psoriasis etc. (Which she publicly talks about naman.) In other words, I always gave her the benefit of the doubt. Di ko na mabilang kung ilang beses nya akong binigo sa lahat ng requests ko sa kanya, kahit pa para sa advocacies. I asked her kahit maikling video message for a Mental Health Awareness Campaign, which she allegedly champions, pero sineenzone nya lang ako. Iniisip ko na lang, baka she just couldn't decide on her own. Lahat daw kasi ng kilos at galaw nya had to be coordinated with her management. Pero kapag sya may kailangan, she would call me, kahit alanganing oras, with matching iyak pa. When she asked me to write a song for Leni, she just edited some of the lyrics and it became hers na. She also asked me to pen the caption for her socmed posts about the song. Pero sa mga press release, she got the credits.

When Moira needs you, she will make you feel loved and cared for. Magaling sya mambola. She even asked me kung magkano price ko for ghostwriting for her last year noong nagkakalabuan na sila ni Jason. When I told her my price, tinawaran nya ako for 20k per month. I felt so insulted. But I just kept that to myself. When I finally had enough May last year, I blocked her in FB and messenger complete with a long breakup letter.

People are telling us, "walang Moira kung wala ang Titibo-tibo." But we never indulged ourselves with that idea. And we maintain that Moira is self-made. She has exceptional gifts and one way or another she'll make it to the top kahit wala ang mga komposisyon namin for her.

Fast-forward tayo sa hiwalayan.

I have been itching to write about this a few months ago pa. Hindi ko na kasi ma-take ang pambabash ng publiko kay Jason. But Jason stopped me because he still wanted to protect Moira. Hindi totoo ang kantang Eme ni Moira na "wala naman akong kasalanan”. Looks and voice can definitely deceive. This soft-spoken lady with a voice of an angel, who always seems to have a sunny disposition, and has a penchant for quoting the Bible, has a dark side unknown to many. (A wolf in sheep's clothing, ika nga.) And I am not surprised sa kung anuman ang narinig ko from those people who saw the whole story (na friends and family din ni Moi). I was strongly advised not to leak any of these. To not even mention it to anyone. But I refuse to be quiet. I refuse to be neutral. To be neutral in times of injustice is to take the side of the oppressor. In this case, Moira is more of an oppressor than a victim. Jason took all the blame, took all the bashing, in the name of love. That's how he was raised. To roll with the punches. To give the other cheek. Jason and Moira are both Christians, pero mukhang si Jason lang ang faithful with the teachings of Christ. Yes, nagkasala sya. Pero walang third party on Jason's side. He never fell in love with anyone else. His only sin was he listened to the call of flesh and availed of illicit massage service because Moira couldn't fulfill his sexual needs. And even before Jason confessed to his sin, Moira already wanted out. Dahil nakahanap na rin sya ng pamalit. Someone who will “dance in the rain” with her and call her “binibini”. I'll leave it to you to guess sino ang tinutukoy ko.

Jason was mocked, wrongly accused and crucified while Moira was busy washing her hands and capitalizing on their breakup by writing awkward songs that only serve her interests and twisted sense of reality. Yes, you can say that Jason is also trying to capitalize on the breakup, but not for clout. It is predicated on his desire to restore the marriage. While EME is mockingly lashing out on Jason, "Ikaw Pa Rin" is a lowkey, heartfelt song that puts Moira on the pedestal of treasured memories. See the difference?

Bilib ako sa lalakeng ito, because he carried on with his faith, he admitted to his sins and faced all the consequences. He did everything to win back Moira despite all the pushback. Samantalang yung isa, busy with grooming her image. Takot malaos. Will do everything and will step on the back of other people in pursuit of clout and longevity.

Despite all the things said and done, I was still hoping for a second chance for Moira and Jason. I've seen uglier and messier break ups. Kami mismo ng asawa ko, we experienced worse. But we held on.

Napatatunayan na ni Jason that he is willing to do his part to restore the marriage. Napatunayan na ni Jason that he can withstand the torrent of insults and hate. He never once defended himself the past year from all the lies made up about him being gay, pumatol sa may asawa, may kabit, manggagamit etc. Fun fact, did you know that 95% of Moira’s hits were composed by Jason? Paubaya, Ikaw at Ako, Pabilin, EDSA, Patawad, Kumpas, Babalik Sayo, Saglit, and the list goes on. Now tell me sino ang manggagamit. He was willing to go down history as the only only bad guy, the only one who made a mistake. But Moira kept singing and releasing defamatory songs against Jason, filling up Araneta Colosseum and touring the world with one purpose for her shows: to get people’s sympathy. So I am breaking my silence so that the truth can finally come out.

Despite what I said against Moira, I cannot discount the fact na nag simula siyang mabuting tao. She's just probably overwhelmed with success and the medications--a fatal combination. I hope she can still find it in her heart to forgive. I hope mamayani sa kanya yung true nature niya na marahil ay nilamon na ng fame and fortune and everything that goes with it. (Sabi nga ni Freddie Mercury.)

Ang pinakapunto ko rito, it's easy for us to root for someone at the expense of another. Sobrang olats talaga ng cancel culture. We hated Jason so much without knowing the backstory. I feel sorry for his family. They didn't deserve to be dragged into this mess. I pray for their healing. At sana lang, this issue can be mended privately, instead of this never-ending trial by publicity.

I'd like to believe that life, much like a symphony, weaves together different melodies and harmonies, creating a composition unique to each individual. Moira and Jason's story, with all its trials and tribulations, serves as a poignant reminder that even amidst the most beautiful melodies, dissonance can arise, threatening to disrupt the harmony we strive to maintain.

In this intricate symphony of life, Moira and Jason were once entwined, their voices blending in perfect harmony. Yet, as the music swelled, they found themselves embarking on separate paths, their melodies drifting apart. But let us not forget that within dissonance lies the potential for growth and transformation. It is through the dissonant chords that we discover the true essence of our being.

As Moira and Jason navigate the aftermath of their separation, may they both find the strength to embrace the dissonance, to delve into the depths of their souls, and emerge with a renewed sense of self. Just as a composer reworks a discordant note to bring forth a harmonious resolution, may Moira and Jason find the courage to mend the fragments of their lives and create a symphony of healing.

In the grand tapestry of existence, let their individual melodies intertwine once more, not as a means to recreate the past, but as a testament to their growth and resilience. May the symphony of their lives continue, enriched by the lessons learned, the wounds healed, and the melodies yet to be discovered.

Addendum: I edited out the "minor offense" part. Kasi I recognize the fact na kasal sila. My apologies.
"""

In [51]:
prompt = statement_2.format("Extract keywords from following statement and create a prompt to create image persona based on the 3rd derived keyword:")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

sentiment_prompt = response["choices"][0]["text"]

print(sentiment_prompt)

image_from_sentiment_url = generate_image(prompt = sentiment_prompt)
get_image(image_url = image_from_sentiment_url)


Keywords: Jason-Moira breakup, mental health condition, benefit of the doubt, illicit massage service, capitalizing on breakup, cancel culture, dissonance, healing, resilience.

Prompt: Create an image persona that conveys the idea of resilience and healing in the aftermath of the Jason-Moira breakup.


In [52]:
save_image(image_url = image_from_sentiment_url, file_location = "data/statement.png")

In [54]:
statement_image = load_raw_image("data/statement.png")
get_prediction(image = statement_image, labels = ["positive economy", "negative economy"])

{'positive economy': 0.08314923942089081,
 'negative economy': 0.9168508052825928}